In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import sklearn.naive_bayes as nb
from sklearn import neighbors 
from sklearn.tree import DecisionTreeClassifier
import pickle
import os
import nltk
from nltk.corpus import stopwords
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import xlsxwriter
import matplotlib.pyplot as plt
import seaborn as sn
from nltk.tokenize import word_tokenize
import itertools

paths to use

In [3]:
write_path = r'C:\Users\matoo\Documents\4th year\final project\some code\Results\Top featurs\\'
read_directory = r'C:\Users\matoo\Documents\4th year\final project\some code\Pickles\concat\test\\'

This function returns the top features for every model

In [4]:
def print_top_features(classifier, fileName, str):
    """Prints features with the highest coefficient values, per class"""
    dfSignificantFeatures = pd.DataFrame()
    class_labels = classifier.lables_
    for i, class_label in enumerate(class_labels):
        top20 = np.argsort(classifier.coef_[i])[-20:]
        toList = (" %s" % (
              ",".join(feature_names[j] for j in top20)))
        list = toList.split(sep=',')

        print("%s: %s" % (class_label,
                          ",".join(feature_names[j] for j in top20)))
        dfSignificantFeatures[class_label.upper()] = list[::-1]
    dfSignificantFeatures.to_csv(write_path+fileName + '_best10Features_'+str+'.csv', mode='a', sep=',')

In [5]:
def calculate_occurencens(df, feature_names):
    langCorpus = np.array(df.Text.values).tolist()
    #tokenize words in tweet
    tokenized_tweets = [word_tokenize(i) for i in langCorpus]
    #merge all tweets
    allwords = [item for sublist in tokenized_tweets for item in sublist]
    allTokens= (itertools.chain.from_iterable(tokenized_tweets))
    resultwords = [word for word in allTokens if word in feature_names]
    tokenFreq = nltk.FreqDist(resultwords)
    ans = len(resultwords)/float(len(allwords))
    return ans

This function returns the train and test sets

In [6]:
def extract_test_set(df, numOfFeatures, RemoveStopWords, n, analyzer):
    stops = set(stopwords.words('english'))
    corpus = np.array(df.Text.values).tolist()
    target = np.array(df['User Language'].values)
    if RemoveStopWords:
        vectorizer = TfidfVectorizer(stop_words='english', max_features=numOfFeatures, analyzer=analyzer, ngram_range=(n, n))
    else: #only SW
        vectorizer = TfidfVectorizer(vocabulary=stops ,max_features=numOfFeatures, analyzer=analyzer, use_idf=False, ngram_range=(n, n))
    
    X = vectorizer.fit_transform(corpus)
    global feature_names
    feature_names = vectorizer.get_feature_names()
    return X, target, feature_names

This function plots a confusion matrix to a .png file

In [7]:
def plot_cnf_matrix(cms, classes, fileName, model_name):
    df_cm = pd.DataFrame(cms, index = classes, columns = classes)
    plt.figure(figsize = (100,100))
    fig, ax = plt.subplots()
    ax = sn.heatmap(df_cm, annot=True, linewidths=.5, fmt='g')
    fig.savefig(write_path + fileName + '_cm_' + model_name +'.png', format='png')
    plt.close(fig)

This function helps us to classify our data 

In [8]:
def Classify(df,fileName,RemoveStopWords,analyzer,n):
    a = {'Classifer': pd.Series(index=['DT', 'Naive bayes' ,'SVM linear']),} #, '% total occur'
    scoreTable = pd.DataFrame(a)

    scoreTable.__delitem__('Classifer')

    tests = ['allLangs'] 
    nums = [500, 750, 1000, 1500, 2000, 3000, 5000, 7000]

    for test in tests:
        dfReduced = df.copy(deep=True)

        if test is not "allLangs":
            langs = str.split(test, '_')
            dfReduced = dfReduced[dfReduced['Lang'].isin(langs)]
        
        #if we want to check for several x top featurs
        for numOfFeatures in nums:
            nfolds=5
            accuracies = []
            file_name = fileName + '_' + str(numOfFeatures) #if we need to change the output file name
            data, target, feat_names = extract_test_set(dfReduced,numOfFeatures,RemoveStopWords, n, analyzer)
            kf = KFold(data.shape[0], n_folds = nfolds, shuffle = True, random_state = 1)
            
            test = test+" top "+str(numOfFeatures)+" features"
            
            dtree = DecisionTreeClassifier(random_state=0, max_depth=50)
            acc = train_model_kf_cv(dtree, kf, data, target, nfolds, file_name, 'DT')
            accuracies.append(acc) 
            
            bayes = nb.MultinomialNB()
            acc = train_model_kf_cv(bayes, kf, data, target, nfolds, file_name, 'Naive bayes')
            accuracies.append(acc)
            
            svm = SVC(decision_function_shape='ovo',kernel='linear')
            acc = train_model_kf_cv(svm, kf, data, target, nfolds, file_name, 'SVM linear')
            accuracies.append(acc)
            
            #knn = neighbors.KNeighborsClassifier()
            #acc = train_model_kf_cv(knn, kf, data, target, nfolds, fileName, 'KNN')
            #accuracies.append(acc)
            
            #occ = calculate_occurencens(df, feat_names)
            #accuracies.append(occ)
            

            scoreTable[file_name] = accuracies
            
    writer = pd.ExcelWriter(write_path + file_name + '_Results.xlsx') 
    scoreTable.to_excel(writer,'Score table')
    writer.save()

This function trains the model k-fold times and returns total accuracy

In [ ]:
def train_model_kf_cv(model, kf, data, target, numFolds, fileName, model_name):
    cm = []
    error = []
    for train_indices, test_indices in kf:
        # Get the dataset; this is the way to access values in a pandas DataFrame
        train_X = data[train_indices, :]
        train_Y = target[train_indices]
        test_X = data[test_indices, :]
        test_Y = target[test_indices]
        # Train the model
        model.fit(train_X, train_Y)
        predictions = model.predict(test_X)
        # Evaluate the model
        classes = model.classes_                
        cm.append(confusion_matrix(test_Y, predictions, labels=classes))
        ###fpr, tpr, _ = roc_curve(test_Y, predictions)
        ###total += auc(fpr, tpr)
        error.append(model.score(test_X, test_Y))
    accuracy = np.mean(error)
    for i in range(0,9):
        cms = np.mean(cm, axis=0)
    plot_cnf_matrix(cms, classes, fileName, model_name)
    ###auc = total / numFolds
    ###print "AUC of {0}: {1}".format(Model.__name__, accuracy)
    return accuracy

main

In [ ]:
lang10 =['bg' 'cs' 'de' 'es' 'fr' 'it' 'ja' 'ko' 'nl' 'ru']
maxsall=[]

pickles = os.listdir(read_directory)
b = {'Data': pd.Series(index=[pickles]),}
SummerayTable = pd.DataFrame(b)

SummerayTable.__delitem__('Data')

count=1
for file in pickles:
    print ("file: "+ file+" num: "+str(count)+" out of " +str(len(pickles)))
    count=count+1
    df = pd.read_pickle(read_directory + str(file))
    feature_names = list(df.columns.values)
    Classify(df, file, True, 'word', 1)  #word or char if character
    #Classify(df, file, True, 'char', 3)  #word or char if character

file: concat_1 num: 1 out of 2
